In [2]:
import os
import pathlib
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import numpy as np
import tritonclient.http as httpclient
from tritonclient.utils import np_to_triton_dtype

In [3]:
from transformers import AutoTokenizer

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def prepare_tensor(name, input):
    client_util = httpclient
    t = client_util.InferInput(name, input.shape,
                               np_to_triton_dtype(input.dtype))
    t.set_data_from_numpy(input)
    return t

In [5]:
triton_url = "localhost:8000"
triton_model_name = "ensemble"
triton_client = httpclient.InferenceServerClient(url=triton_url)

In [24]:
input0 = [["translate English to German: This is good"], ["translate English to German: I feel happy today after so many days"]]
input0_data = np.array(input0).astype(object)
max_new_tokens = 50
input1_data = np.ones_like(input0).astype(np.int32) * max_new_tokens
inputs = [
    prepare_tensor("text_input", input0_data),
    prepare_tensor("max_new_tokens", input1_data)
]

outputs = []
outputs.append(httpclient.InferRequestedOutput("text_output"))
results = triton_client.infer(model_name=triton_model_name,
                              inputs=inputs,
                              outputs=outputs)

outputs = results.as_numpy('text_output')
outputs = [o.decode() for o in outputs]
print(outputs)

['Das ist gut.', 'Ich bin heute glücklich, nach so vielen Tagen']
